# Coupled higgs equation 
$$
u_{tt} - u_{xx} + |u|^2 u - 2uv = 0
$$
$$
v_{tt} + v_{xx} - (\left| u \right|^2)_{xx} = 0
$$

where, $ u(x,t) $ represents a complex nucleon field and $ v(x,t) $ represents a real scalar meson field. The coupled Higgs field Equation describes a system of conserved scalar nucleon interaction with a neutral scalar meson.

solutions 

$$
u_1(x, t) = ir e^{ir(\omega x + t)} \sqrt{1 + \omega^2} \tanh\left(\frac{r(k + x + \omega t)}{\sqrt{2}}\right)
$$
$$
v_1(x, t) = r^2 \tanh^2\left(\frac{r(k + x + \omega t)}{\sqrt{2}}\right)
$$

for $t = 0$

$$
u_1(x, 0) = ir e^{ir \omega x} \sqrt{1 + \omega^2} \tanh\left(\frac{r(k + x)}{\sqrt{2}}\right)
$$
$$
v_1(x, 0) = r^2 \tanh^2\left(\frac{r(k + x)}{\sqrt{2}}\right)
$$

where 
$k = 4, \omega = 5 , \alpha = 2, c = 2, r = 2$


In [5]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

class SinActv(nn.Module):
    def forward(self, input_):
        return torch.sin(input_)

class ModelU1(nn.Module):
    def __init__(self, n_input_units=2, shared_units=128, branch_units=32, n_output_units=1, n_shared_layers=4, n_branch_layers=1, actv=nn.Tanh):
        super(ModelU1, self).__init__()
        # Initial shared layers setup
        self.shared_layers = nn.ModuleList()
        for i in range(n_shared_layers):
            if i == 0:
                self.shared_layers.append(nn.Linear(n_input_units, shared_units))
            else:
                self.shared_layers.append(nn.Linear(shared_units, shared_units))
            self.shared_layers.append(actv())

        # Branch for u_real
        self.branch_u_r_layers = self._make_branch_layers(shared_units, branch_units, n_output_units, n_branch_layers, actv)

        # Branch for u_imag
        self.branch_u_i_layers = self._make_branch_layers(shared_units, branch_units, n_output_units, n_branch_layers, actv)

    def _make_branch_layers(self, in_units, units_per_layer, n_output_units, n_layers, actv):
        layers = nn.ModuleList()
        for i in range(n_layers):
            layers.append(nn.Linear(in_units, units_per_layer))
            layers.append(actv())
            in_units = units_per_layer
        layers.append(nn.Linear(in_units, n_output_units))
        return layers

    def _forward_branch(self, shared, layers):
        x = shared
        for layer in layers:
            x = layer(x)
        return x

    def forward(self, x):
        shared = x
        for layer in self.shared_layers:
            shared = layer(shared)
        u_r = self._forward_branch(shared, self.branch_u_r_layers)
        u_i = self._forward_branch(shared, self.branch_u_i_layers)
        return u_r, u_i

class ModelV1(nn.Module):
    def __init__(self, n_input_units=2, shared_units=128, n_output_units=1, n_layers=5, actv=nn.Tanh):
        super(ModelV1, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(n_layers):
            if i == 0:
                self.layers.append(nn.Linear(n_input_units, shared_units))
            else:
                self.layers.append(nn.Linear(shared_units, shared_units))
            self.layers.append(actv())
        # Output layer for v
        self.layers.append(nn.Linear(shared_units, n_output_units))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def grad(outputs, inputs):
    return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True, retain_graph=True)[0]

def laplacian(field, x, t):
    field_x = grad(field, x)
    field_xx = grad(field_x, x)
    field_t = grad(field, t)
    field_tt = grad(field_t, t)
    return field_xx, field_tt

# Define the ODE system for the Coupled Higgs field equations
def coupled_higgs(u_real, u_imag, v, x, t):
    u_r_xx, u_r_tt = laplacian(u_real, x, t)
    u_i_xx, u_i_tt = laplacian(u_imag, x, t)
    v_xx, v_tt = laplacian(v, x, t)

    u_abs = u_real**2 + u_imag**2
    u_abs_xx, u_abs_tt = laplacian(u_abs, x, t)

    # Calculate the field equations
    du_eq_r = u_r_tt - u_r_xx + u_abs * u_real - 2 * u_real * v
    du_eq_i = u_i_tt - u_i_xx + u_abs * u_imag - 2 * u_imag * v
    dv_eq = v_tt + v_xx - u_abs_xx
    
    return du_eq_r, du_eq_i, dv_eq

# Function to calculate the real part of u1
def real_u1(x, t, k, omega, r):
    return np.real(1j * r * np.exp(1j * r * (omega * x + t)) * np.sqrt(1 + omega**2) *
                   np.tanh((r * (k + x + omega * t)) / np.sqrt(2)))

def imag_u1(x, t, k, omega, r):
    return np.imag(1j * r * np.exp(1j * r * (omega * x + t)) * np.sqrt(1 + omega**2) *
                   np.tanh((r * (k + x + omega * t)) / np.sqrt(2)))
    
def real_v1(x, t, k, omega, r):
    return (r * np.tanh((r * (k + x + omega * t)) / np.sqrt(2)) )**2

In [6]:
# Check if CUDA is available and set the default device
if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

model_u1 = ModelU1(n_input_units=2, shared_units=128, branch_units=32, n_output_units=1, n_shared_layers=4, n_branch_layers=1, actv=nn.Tanh).to(device)
model_v1 = ModelV1(n_input_units=2, shared_units=128, n_output_units=1, n_layers=5, actv=nn.Tanh).to(device)

num_epochs = 100000  # Number of training epochs
lr = 1e-3          # Learning rate
num_samples = 1000 # Number of samples for training
k = 4
omega = 5
r = 2

optimizer_u1 = Adam(model_u1.parameters(), lr=lr)
optimizer_v1 = Adam(model_v1.parameters(), lr=lr)
mse_cost_function = torch.nn.MSELoss()
model_save_path = 'model_weights'
os.makedirs(model_save_path, exist_ok=True)

CUDA is available! Training on GPU.


In [ ]:
# Training loop
for epoch in tqdm(range(num_epochs),desc='Progress:', leave=True, ncols=75, mininterval=0.1, ascii=False):
    # Generate random samples for x and t
    x = (torch.rand(num_samples, 1) * 2 - 5).to(device)  # x in range [-5, -3]
    x.requires_grad = True
    t = (torch.rand(num_samples, 1) * 1).to(device)      # t in range [0, 1]
    t.requires_grad = True
    x_t = torch.cat([x, t], dim=1) 
    
    optimizer_u1.zero_grad()
    pred_u_r, pred_u_i = model_u1(x_t)
    optimizer_v1.zero_grad()
    pred_v = model_v1(x_t)
    #pred_u_r, pred_u_i, pred_v = shared_model(x_t)
    du_eq_r, du_eq_i, dv_eq = coupled_higgs(pred_u_r, pred_u_i, pred_v, x, t)
    loss_pde_u = torch.mean(du_eq_r**2 + du_eq_i**2)  # Loss for U1 model components
    loss_pde_v = torch.mean(dv_eq**2)  # Loss for V1 model component

    # Calculate the boundary loss at t=0
    x_bc = np.random.uniform(-5, -3, (num_samples, 1))
    t_bc = np.zeros((num_samples, 1))
    real_u1_t0_val = torch.tensor(real_u1(x_bc, t_bc, k, omega, r), device=device).float().view(-1, 1)
    imag_u1_t0_val = torch.tensor(imag_u1(x_bc, t_bc, k, omega, r), device=device).float().view(-1, 1)
    real_v1_t0_val = torch.tensor(real_v1(x_bc, t_bc, k, omega, r), device=device).float().view(-1, 1)
    x_t0 = Variable(torch.from_numpy(np.hstack([x_bc, t_bc])).float(), requires_grad=False).to(device)
    #pred_u_r0, pred_u_i0, pred_v0 = shared_model(x_t0)
    pred_u_r0, pred_u_i0 = model_u1(x_t0) 
    pred_v0 = model_v1(x_t0)
    boundary_loss_ur0 = mse_cost_function(pred_u_r0, real_u1_t0_val)
    boundary_loss_ui0 = mse_cost_function(pred_u_i0, imag_u1_t0_val)
    
    boundary_loss_v_t0 = mse_cost_function(pred_v0, real_v1_t0_val)
    boundary_loss_u_t0 = torch.mean(boundary_loss_ur0 + boundary_loss_ui0)
    # boundary cond for x = -5 and x = -3 

    # Calculate the boundary loss at x = -5
    t_bc_5 = np.random.uniform(0, 1, (num_samples, 1))
    x_bc_5 = np.ones((num_samples, 1))*(-5)
    real_u1_x_val_5 = torch.tensor(real_u1(x_bc_5, t_bc_5, k, omega, r), device=device).float().view(-1, 1)
    imag_u1_x_val_5 = torch.tensor(imag_u1(x_bc_5, t_bc_5, k, omega, r), device=device).float().view(-1, 1)
    real_v1_x_val_5 = torch.tensor(real_v1(x_bc_5, t_bc_5, k, omega, r), device=device).float().view(-1, 1)
    x_t5 = Variable(torch.from_numpy(np.hstack([x_bc_5, t_bc_5])).float(), requires_grad=False).to(device)
    #pred_u_r_x_5, pred_u_i_x_5, pred_v_x_5 = shared_model(x_t5)
    pred_u_r_x_5, pred_u_i_x_5 = model_u1(x_t5)
    pred_v_x_5 = model_v1(x_t5)
    boundary_loss_ur_x5 = mse_cost_function(pred_u_r_x_5, real_u1_x_val_5)
    boundary_loss_ui_x5 = mse_cost_function(pred_u_i_x_5, imag_u1_x_val_5)
    
    boundary_loss_v_t_x5 = mse_cost_function(pred_v_x_5, real_v1_x_val_5)
    boundary_loss_u_t_x5 = torch.mean(boundary_loss_ur_x5 + boundary_loss_ui_x5)

    # boundary loss at x = -4
    t_bc_4 = np.random.uniform(0, 1, (num_samples, 1))
    x_bc_4 = np.ones((num_samples, 1))*(-4)
    real_u1_x_val_4 = torch.tensor(real_u1(x_bc_4, t_bc_4, k, omega, r), device=device).float().view(-1, 1)
    imag_u1_x_val_4 = torch.tensor(imag_u1(x_bc_4, t_bc_4, k, omega, r), device=device).float().view(-1, 1)
    real_v1_x_val_4 = torch.tensor(real_v1(x_bc_4, t_bc_4, k, omega, r), device=device).float().view(-1, 1)
    x_t4 = Variable(torch.from_numpy(np.hstack([x_bc_4, t_bc_4])).float(), requires_grad=False).to(device)
    pred_u_r_x_4, pred_u_i_x_4 = model_u1(x_t4)
    pred_v_x_4 = model_v1(x_t4)
    boundary_loss_ur_x4 = mse_cost_function(pred_u_r_x_4, real_u1_x_val_4)
    boundary_loss_ui_x4 = mse_cost_function(pred_u_i_x_4, imag_u1_x_val_4)
    
    boundary_loss_v_t_x4 = mse_cost_function(pred_v_x_4, real_v1_x_val_4)
    boundary_loss_u_t_x4 = torch.mean(boundary_loss_ur_x4 + boundary_loss_ui_x4)
    
    # Calculate the boundary loss at x = -3
    t_bc_3 = np.random.uniform(0, 1, (num_samples, 1))
    x_bc_3 = np.ones((num_samples, 1))*(-3)
    real_u1_x_val_3 = torch.tensor(real_u1(x_bc_3, t_bc_3, k, omega, r), device=device).float().view(-1, 1)
    imag_u1_x_val_3 = torch.tensor(imag_u1(x_bc_3, t_bc_3, k, omega, r), device=device).float().view(-1, 1)
    real_v1_x_val_3 = torch.tensor(real_v1(x_bc_3, t_bc_3, k, omega, r), device=device).float().view(-1, 1)
    x_t3 = Variable(torch.from_numpy(np.hstack([x_bc_3, t_bc_3])).float(), requires_grad=False).to(device)
    pred_u_r_x_3, pred_u_i_x_3 = model_u1(x_t3)
    pred_v_x_3 = model_v1(x_t3)
    boundary_loss_ur_x3 = mse_cost_function(pred_u_r_x_3, real_u1_x_val_3)
    boundary_loss_ui_x3 = mse_cost_function(pred_u_i_x_3, imag_u1_x_val_3)
    
    boundary_loss_v_t_x3 = mse_cost_function(pred_v_x_3, real_v1_x_val_3)
    boundary_loss_u_t_x3 = torch.mean(boundary_loss_ur_x3 + boundary_loss_ui_x3)

    # Total loss 
    loss_u = loss_pde_u + boundary_loss_u_t0 + boundary_loss_u_t_x5 + boundary_loss_u_t_x4 + boundary_loss_u_t_x3
    loss_v = loss_pde_v + boundary_loss_v_t0 + boundary_loss_v_t_x5 + boundary_loss_v_t_x4 + boundary_loss_v_t_x3

    # Backward pass and optimize
    
    #loss_u.backward(retain_graph=True)
    #optimizer_u1.step()

    loss_v.backward()
    optimizer_v1.step()
    
    # Print loss every few epochs
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss U: {loss_u.item()} , Loss V: {loss_v.item()}')
        model_u1_filename = os.path.join(model_save_path, f'C_HIGGS_U1_epoch_{epoch}.pth')
        torch.save(model_u1.state_dict(), model_u1_filename)
        
        model_v1_filename = os.path.join(model_save_path, f'C_HIGGS_V1_epoch_{epoch}.pth')
        torch.save(model_v1.state_dict(), model_v1_filename)

CUDA is available! Training on GPU.


Progress::   0%|                      | 3/100000 [00:00<2:30:55, 11.04it/s]

Epoch 0, Loss U: 308.2165222167969 , Loss V: 39.47106170654297


Progress::   0%|                     | 72/100000 [00:04<1:37:54, 17.01it/s]


KeyboardInterrupt: 